In [1]:
import math
import pandas as pd
import geopandas as gpd
from datetime import datetime
import rioxarray as rxr
import xarray as xr
from geopandas import points_from_xy
import os
from shapely.geometry import Point
import numpy as np
import dask
from dask.diagnostics import ProgressBar

### HRMET function for python

In [15]:
def HRMET_py(LAI, h, Tsurf, lon, lat,
             datetime_, daylightSavings,
             Tair, SWin, u, ea, pa, Zair, Zu,
             albSoil, albVeg, emissSoil, emissVeg):
    """
    Pure-Python port of HRMET (Zipper & Loheide, 2014).
    datetime_: datetime.datetime
    daylightSavings: 1 or 0
    lon, lat: decimal degrees
    All others: floats
  # Necessary inputs are:
  # (1) Site information:
  #      -datetime = Date & time in Date format
  #      -daylightSavings = daylight savings time? [1=yes (summer), 0=no]
  #      -longitude = Longitude [decimal degrees]
  #      -latitude = Latitude [decimal degrees]
  #      -albSoil = Albedo of soil [0-1]
  #      -albVeg = Albedo of vegetation [0-1]
  #      -emissSoil = Emissivity of soil [0-1]
  #      -emissVeg = Emissivity of vegetation [0-1]
  # (2) Meteorological data:
  #      -Tair = Air temperature, degrees celsius
  #      -SWin = Incoming shortwave radiation [W/m2]
  #      -u = Wind speed [m/s]
  #      -ea = Air vapor pressure [kPa] (can be calculated from Tair and
  #            relative humidity)
  #      -pa = Atmospheric pressure [kPa]
  #      -Zair = height of air temperature measurements [m]
  #      -Zu = height of wind speed measurements [m]
  # (3) Canopy structure data:
  #      -LAI = Leaf Area Index [m2/m2]
  #      -h = Canopy height [m]
  # (4) Canopy surface temperature:
  #      -Tsurf = Canopy surface temperature [deg C]
    
    
    Returns ET [mm/hr]
    """
    # ─── helpers ─────────────────────────────
    def sind(x): return math.sin(math.radians(x))
    def cosd(x): return math.cos(math.radians(x))
    def asind(x): return math.degrees(math.asin(x))
    def acosd(x): return math.degrees(math.acos(x))
    
    # ─── 1️⃣ INPUT CHECKS & FLOOR VALUES ──────
    args = [datetime_, lon, lat, Tair, SWin, u, ea, pa,
            Zair, Zu, LAI, h, Tsurf, albSoil, albVeg, emissSoil, emissVeg]
    for v in args:
        if v is None or (isinstance(v, float) and math.isnan(v)):
            return np.nan
    if Zu < h:
        raise ValueError("Zu < h; wind sensor below canopy")
    if Zair < h:
        raise ValueError("Zair < h; air‐temp sensor below canopy")
    u   = max(u,   0.1)
    ea  = max(ea,  0.01)
    LAI = max(LAI, 0.01)
    h   = max(h,   0.01)

    # ─── 2️⃣ CONSTANTS & TEMPERATURES ─────────
    stefBoltz = 5.67e-8          # [W m-2 K-4] - Stefan-Boltzmann Constant
    cP        = 29.3             # [J mol-1 degC-1] - specific heat of air, from Campbell & Normal Table A1
    mW        = 0.018            # [kg mol-1] - molecular mass of water, from C&N table A1
    mA        = 0.029            # [kg mol-1] - molecular mass of air, from C&N table A1
    rhoW      = 1000             # [kg m-3] - density of water, from C&N table A2
    Tair_K    = Tair + 273.16    # [degK] - air temperature, in Kelvin
    Tsurf_K   = Tsurf + 273.16   # [degK] - surface temperature
    rhoA      = (44.6 * pa * 273.15) / (101.3 * Tair_K)  # molar density air, C&N eq. 3.3
    k         = 0.4              # [-] - von Karman's constant
    # Saturation vapor curve constants (if you need them later) constant 'a' for es equations, C&N pg 41
    # esA, esB, esC = 0.611, 17.502, 240.97
    
    # ─── 3️⃣ JULIAN DAY & TIME ─────────────────
    julianDay  = datetime_.timetuple().tm_yday
    julianTime = datetime_.hour + datetime_.minute/60.0

    # ─── 4️⃣ SOLAR GEOMETRY ────────────────────
    f      = 279.575 + 0.9856 * julianDay
    eqTime = (1/3600) * (
        -104.7*sind(f) +
         596.2*sind(2*f) +
           4.3*sind(3*f) -
          12.7*sind(4*f) -
         429.3*cosd(f) -
           2  *cosd(2*f) +
          19.3*cosd(3*f)
    )
    # longitude correction to standard meridian
    lon_corr = lon
    while lon_corr > 7.5:
        lon_corr -= 15
    LC        = -lon_corr / 15.0
    solarNoon = 12 + daylightSavings - LC - eqTime
    solarDec  = asind(0.39785 * sind(278.97 + 0.9856*julianDay +
                    1.9165*sind(356.6 + 0.9856*julianDay)))
    zenith    = acosd(
        sind(lat)*sind(solarDec) +
        cosd(lat)*cosd(solarDec)*cosd(15*(julianTime-solarNoon))
    )

    # ─── 5️⃣ DOWNWELLING LONGWAVE (LWin) ───────
    solConst = 1361.5  # W m⁻²
    m_air    = 35 * cosd(zenith) * (1224*cosd(zenith)**2 + 1)**(-0.5)
    # lookup G_Tw by lat band (Smith 1966 Table 1)
    if   lat < 10:  G_Tw=2.80
    elif lat < 20:  G_Tw=2.70
    elif lat < 30:  G_Tw=2.98
    elif lat < 40:  G_Tw=2.92
    elif lat < 50:  G_Tw=2.77
    elif lat < 60:  G_Tw=2.67
    elif lat < 70:  G_Tw=2.61
    elif lat < 80:  G_Tw=2.24
    elif lat < 90:  G_Tw=1.94
    else: raise ValueError("Latitude > 90°")
    # dewpoint (Crawford & Duchon, 1999)
    Tdew      = 240.97 * math.log(ea/0.611) / (17.502 - math.log(ea/0.611))
    atmWater  = math.exp(0.1133 - math.log(G_Tw+1) + 0.0393*Tdew)
    TrTpg     = 1.021 - 0.084*(m_air*(0.000949*pa*10+0.051))**0.5
    Tw        = 1 - 0.077*(atmWater*m_air)**0.3
    Ta        = 0.935**m_air
    Rso       = solConst * cosd(zenith) * TrTpg * Tw * Ta
    clf       = max(0, min(1, 1 - SWin/Rso))
    emissSky  = (
        clf +
        (1-clf)*(1.22+0.06*math.sin((datetime_.month+2)*math.pi/6)) *
        (ea*10/Tair_K)**(1/7)
    )
    LWin      = emissSky * stefBoltz * Tair_K**4

    # ─── 6️⃣ UPWELLING LW & SW ─────────────────
    fc        = 1 - math.exp(-0.5*LAI)
    LWoutVeg  = emissVeg * stefBoltz * Tsurf_K**4 * (1 - math.exp(0.9*math.log(1-fc)))
    LWoutSoil = emissSoil * stefBoltz * Tsurf_K**4 *  math.exp(0.9*math.log(1-fc))
    LWout     = LWoutVeg + LWoutSoil
    SWoutVeg  = SWin * (1 - math.exp(0.9*math.log(1-fc))) * albVeg
    SWoutSoil = SWin * math.exp(0.9*math.log(1-fc)) * albSoil
    SWout     = SWoutVeg + SWoutSoil

    # ─── 7️⃣ NET R & G ────────────────────────
    R_net = SWin - SWout + LWin - LWout
    G     = 0.35 * R_net * math.exp(0.9*math.log(1-fc))

    # ─── 8️⃣ ROUGHNESS & DISP PLATE ───────────
    k       = 0.4
    Cw, Cr, Cs, Cd1, uMax = 2, 0.3, 0.003, 7.5, 0.3
    subRough= math.log(Cw)-1+(1/Cw)
    u_uh    = min(uMax, math.sqrt(Cs + Cr*(LAI/2)))
    d       = h*(1 - (1-math.exp(-math.sqrt(Cd1*LAI)))/math.sqrt(Cd1*LAI))
    z0m     = h*(1-d/h)*math.exp(-k*(1/u_uh)-subRough)
    kB1     = 2.3
    z0h     = z0m / math.exp(kB1)

    # ─── 9️⃣ ITERATIVE SENSIBLE HEAT: stable & unstable ─────────
    def iterate_H(zeta_init, H_init):
        zeta, Hiter, change = zeta_init, H_init, 1.0
        i = 0
        while abs(change) > 0.001:
            i += 1
            Hprev = Hiter
            if zeta > 0:
                diabM = 6 * math.log(1+zeta)
                diabH = diabM
            else:
                diabH = -2 * math.log((1 + math.sqrt(1-16*zeta)) / 2)
                diabM = 0.6 * diabH
            uStar = u * k / (math.log((Zu-d)/z0m) + diabM)
            rHa   = 1 / (k**2 * u * rhoA / (
                      1.4 * (math.log((Zu-d)/z0m)+diabM) *
                            (math.log((Zair-d)/z0h)+diabH)
                    ))
            rEx  = mA * math.log(z0m/z0h) / (rhoA * k * uStar)
            rHtot= rHa + rEx
            Hiter = cP*(Tsurf - Tair)/rHtot
            zeta  = -k * 9.8 * Zu * Hiter / (rhoA * cP * Tair_K * uStar**3)
            change= (Hiter-Hprev)/Hprev if i>1 else 1.0
            if i>10000:
                raise RuntimeError("Sensible-heat iteration did not converge")
        return max(Hiter, 0.02)
    
    H_low  = iterate_H( zeta_init= 0.5, H_init= 0.5)
    H_high = iterate_H( zeta_init=-0.5, H_init=-0.5)
    if abs(H_low-H_high)/H_low <= 0.01:
        H = 0.5*(H_low + H_high)
    else:
        raise RuntimeError("H_low and H_high diverged")
    
    # ─── 🔟 LATENT HEAT & ET ─────────────────────
    LE     = R_net - G - H
    lamda  = (2.495 - 2.36e-3*Tsurf) * mW * 1e6
    ET     = (mW/rhoW) * 3600 * 1000 * LE/lamda
    return max(ET, 0.0)

# Data prepration

### Constant HRMET arguments

In [3]:
# Tair, SWin, u, ea, pa, Zair, Zu, albSoil, albVeg, emissSoil, emissVeg
#      -albSoil = Albedo of soil [0-1]
  #      -albVeg = Albedo of vegetation [0-1]
  #      -emissSoil = Emissivity of soil [0-1]
  #      -emissVeg = Emissivity of vegetation [0-1]
  # (2) Meteorological data:
  #      -Tair = Air temperature, degrees celsius
  #      -SWin = Incoming shortwave radiation [W/m2]
  #      -u = Wind speed [m/s]
  #      -ea = Air vapor pressure [kPa] (can be calculated from Tair and
  #            relative humidity)
  #      -pa = Atmospheric pressure [kPa]
  #      -Zair = height of air temperature measurements [m]
  #      -Zu = height of wind speed measurements [m]

In [4]:
# Constants from literature
albSoil = 0.105
albVeg = 0.2
emissSoil = 0.945
emissVeg = 0.94

In [5]:
# Eddy covariance tower info (US-UC1) Meteorological data
EC_tower_data = pd.read_csv(os.path.join(os.getcwd(), 'Data', 'EC_tower', 'USUC1_df_2022_ETa_MET.csv'),
                            parse_dates = ['TIMESTAMP_START'],
                           index_col ='TIMESTAMP_START' )

In [6]:
EC_tower_data.loc['2022-08-03']

,TA_1_1_1,RH_1_1_1,SW_IN_1_1_1,LW_IN_1_1_1,WS,PA,P_RAIN_1_1_1,ETa_corr
TIMESTAMP_START,,,,,,,,
2022-08-03 06:00:00,15.04075,93.45115,186.18450,332.3865,0.264218,97.82250,0.0,NaN
2022-08-03 07:00:00,18.88665,80.86965,385.75600,331.3590,0.886605,97.86890,0.0,0.200235
2022-08-03 08:00:00,21.92335,65.80670,579.41700,345.1780,1.392615,97.89430,0.0,0.288460
2022-08-03 09:00:00,24.21535,61.58120,742.53900,359.3660,1.573360,97.92745,0.0,0.359319
2022-08-03 10:00:00,26.03365,59.45000,860.58000,371.6690,1.277975,97.93175,0.0,0.398956
2022-08-03 11:00:00,27.41110,56.54630,926.24500,381.5190,1.669470,97.89325,0.0,0.484754
2022-08-03 12:00:00,28.75035,51.70310,965.04750,389.3770,2.040650,97.86520,0.0,0.459087
2022-08-03 13:00:00,29.84720,48.33435,929.07700,396.9680,2.378745,97.82345,0.0,0.512379
2022-08-03 14:00:00,30.56625,46.76415,833.08350,399.1885,2.164280,97.75225,0.0,0.486164


In [7]:
datetime_ = datetime.strptime('2022-08-03 13:00:00', "%Y-%m-%d %H:%M:%S")
Tair = EC_tower_data.loc['2022-08-03 13:00:00']['TA_1_1_1']
SWin = EC_tower_data.loc['2022-08-03 13:00:00']['SW_IN_1_1_1']
u    = EC_tower_data.loc['2022-08-03 13:00:00']['WS']
RH   = EC_tower_data.loc['2022-08-03 13:00:00']['RH_1_1_1']
esat = 0.6108 * np.exp(17.27 * Tair / (Tair + 237.3))   # saturation vapor pressure (kPa)
ea   = (RH / 100.0) * esat
pa   = EC_tower_data.loc['2022-08-03 13:00:00']['PA']
# based on measurement height from ameriflux website
Zair = 3
Zu   = 5

In [34]:
# determine the Rn constant to convert hourly to daily ETa
EC_tower_data.loc['2022-08-03']['SW_IN_1_1_1'].sum()/EC_tower_data.loc['2022-08-03 13:00:00']['SW_IN_1_1_1']

np.float64(8.637691439999053)

### Variable HRMET arguments

In [5]:
# read LAI
LAI_raster = rxr.open_rasterio(os.path.join(os.getcwd(), 'Data', 'LAI',
                                            "LTARgatesburg_2022Aug03_AltumSR_RGBnirRE_WGS84_UTM17N_4_Berni_et_al_2009.tif"),
                               masked=True).squeeze("band", drop=True).rename("LAI")

# read CHM
CHM_raster = rxr.open_rasterio(os.path.join(os.getcwd(), 'Data', 'Lidar_CHM_2022',
                                            "LTARgatesburg_combined2022Flights_5cmBL_CHM_WGS84_UTM17N.tif"),
                               masked=True).squeeze("band", drop=True).rename("CHM")

# read LST
LST_raster = rxr.open_rasterio(os.path.join(os.getcwd(), 'Data', 'Thermal_Imagery',
                                            "Gatesburg_3Aug2022_Altum_ThermalC_WGS84_UTM_clip.tif"),
                               masked=True).squeeze("band", drop=True).rename("LST")

#### Aligning rasters to LST

In [6]:
# 1️⃣ Same CRS?
print(LAI_raster.rio.crs, CHM_raster.rio.crs, LST_raster.rio.crs)

# 2️⃣ Same pixel resolution?
print("res1:", LAI_raster.rio.resolution(), "res2:", CHM_raster.rio.resolution(), "res3:", LST_raster.rio.resolution())

# 3️⃣ Same affine transform (origin + rotation)?
print("transform1:", LAI_raster.rio.transform(), 
      "\ntransform2:", CHM_raster.rio.transform(), 
      "\ntransform3:", LST_raster.rio.transform())

# 4️⃣ Same coordinate vectors?
print("x coords equal?", LAI_raster.x.equals(CHM_raster.x))
print("y coords equal?", LAI_raster.y.equals(CHM_raster.y))

EPSG:32617 EPSG:32617 EPSG:32617
res1: (0.05000000000000358, -0.05) res2: (0.05000000000001006, -0.04999999999998979) res3: (0.060302688957139326, -0.060302688957139326)
transform1: | 0.05, 0.00, 752427.88|
| 0.00,-0.05, 4516352.28|
| 0.00, 0.00, 1.00| 
transform2: | 0.05, 0.00, 752488.88|
| 0.00,-0.05, 4516367.33|
| 0.00, 0.00, 1.00| 
transform3: | 0.06, 0.00, 752465.20|
| 0.00,-0.06, 4516371.21|
| 0.00, 0.00, 1.00|
x coords equal? False
y coords equal? False


In [40]:
for p in paths:
    with rasterio.open(p) as src:
        # read as a masked array so nodata → mask=True
        arr = src.read(1, masked=True)
        
        total_pixels = src.width * src.height
        nodata_pixels = np.count_nonzero(arr.mask)
        valid_pixels  = total_pixels - nodata_pixels
        
        print(f"{p}:")
        print(f"  Total pixels : {total_pixels}")
        print(f"  Valid pixels : {valid_pixels}")
        print(f"  Nodata pixels: {nodata_pixels}")
        print("---")

C:\Users\adadkhah\OneDrive - University of Vermont\UVM\Projects\ET\Hourly_predictions_ffp_train\UAV_ET\Data\LAI\LTARgatesburg_2022Aug03_AltumSR_RGBnirRE_WGS84_UTM17N_4_Berni_et_al_2009.tif:
  Total pixels : 390960360
  Valid pixels : 232177080
  Nodata pixels: 158783280
---
C:\Users\adadkhah\OneDrive - University of Vermont\UVM\Projects\ET\Hourly_predictions_ffp_train\UAV_ET\Data\Lidar_CHM_2022\LTARgatesburg_combined2022Flights_5cmBL_CHM_WGS84_UTM17N.tif:
  Total pixels : 295522441
  Valid pixels : 131468400
  Nodata pixels: 164054041
---
C:\Users\adadkhah\OneDrive - University of Vermont\UVM\Projects\ET\Hourly_predictions_ffp_train\UAV_ET\Data\Thermal_Imagery\Gatesburg_3Aug2022_Altum_ThermalC_WGS84_UTM_clip.tif:
  Total pixels : 209645488
  Valid pixels : 94018389
  Nodata pixels: 115627099
---


In [47]:
import rioxarray as rxr

paths = [os.path.join(os.getcwd(), 'Data', 'LAI',"LTARgatesburg_2022Aug03_AltumSR_RGBnirRE_WGS84_UTM17N_4_Berni_et_al_2009.tif"),
         os.path.join(os.getcwd(), 'Data', 'Lidar_CHM_2022', "LTARgatesburg_combined2022Flights_5cmBL_CHM_WGS84_UTM17N.tif"),
         os.path.join(os.getcwd(), 'Data', 'Thermal_Imagery', "Gatesburg_3Aug2022_Altum_ThermalC_WGS84_UTM_clip.tif")]

# 1️⃣ Load your “master” raster (we’ll use raster1) (LST)
master = (
    rxr.open_rasterio(paths[2], masked=True)
       .squeeze("band", drop=True)
       .rio.set_crs("EPSG:32617")         # ensure the right CRS
       # you can also force a custom resolution here:
       # .rio.reproject("EPSG:32617", resolution=(10,10))
)

# 2️⃣ Load & align the other two to that exact grid
aligned = []
for path in paths[0:3]:
    da = (
        rxr.open_rasterio(path, masked=True)
           .squeeze("band", drop=True)
           .rio.set_crs("EPSG:32617")
           .rio.reproject_match(master)
    )
    aligned.append(da)

C:\Users\adadkhah\AppData\Local\Temp\ipykernel_17376\3288984772.py:11: FutureWarning: It is recommended to use 'rio.write_crs()' instead. 'rio.set_crs()' will likelybe removed in a future release.
  .rio.set_crs("EPSG:32617")         # ensure the right CRS
C:\Users\adadkhah\AppData\Local\Temp\ipykernel_17376\3288984772.py:22: FutureWarning: It is recommended to use 'rio.write_crs()' instead. 'rio.set_crs()' will likelybe removed in a future release.
  .rio.set_crs("EPSG:32617")
C:\Users\adadkhah\AppData\Local\Temp\ipykernel_17376\3288984772.py:22: FutureWarning: It is recommended to use 'rio.write_crs()' instead. 'rio.set_crs()' will likelybe removed in a future release.
  .rio.set_crs("EPSG:32617")
C:\Users\adadkhah\AppData\Local\Temp\ipykernel_17376\3288984772.py:22: FutureWarning: It is recommended to use 'rio.write_crs()' instead. 'rio.set_crs()' will likelybe removed in a future release.
  .rio.set_crs("EPSG:32617")


In [48]:
arrays = aligned
names  = ["raster1", "raster2", "raster3"]

# 1) CRS
print("CRSs:")
for name, da in zip(names, arrays):
    print(f"  {name}: {da.rio.crs}")

# 2) Shape
print("\nShapes (y, x):")
for name, da in zip(names, arrays):
    print(f"  {name}: {da.shape}")

# 3) Transform
print("\nAffines:")
for name, da in zip(names, arrays):
    print(f"  {name}: {da.rio.transform()}")

# 4) Resolution
print("\nResolutions:")
for name, da in zip(names, arrays):
    print(f"  {name}: {da.rio.resolution()}")

# 5) x / y coords equality
print("\nCoordinate array matches:")
for name, da in zip(names[1:], arrays[1:]):
    same_x = arrays[0].x.equals(da.x)
    same_y = arrays[0].y.equals(da.y)
    print(f"  {name}: x match? {same_x},  y match? {same_y}")

CRSs:
  raster1: EPSG:32617
  raster2: EPSG:32617
  raster3: EPSG:32617

Shapes (y, x):
  raster1: (15184, 13807)
  raster2: (15184, 13807)
  raster3: (15184, 13807)

Affines:
  raster1: | 0.06, 0.00, 752465.20|
| 0.00,-0.06, 4516371.21|
| 0.00, 0.00, 1.00|
  raster2: | 0.06, 0.00, 752465.20|
| 0.00,-0.06, 4516371.21|
| 0.00, 0.00, 1.00|
  raster3: | 0.06, 0.00, 752465.20|
| 0.00,-0.06, 4516371.21|
| 0.00, 0.00, 1.00|

Resolutions:
  raster1: (0.060302688957138174, -0.06030268895715281)
  raster2: (0.060302688957138174, -0.06030268895715281)
  raster3: (0.060302688957138174, -0.06030268895715281)

Coordinate array matches:
  raster2: x match? True,  y match? True
  raster3: x match? True,  y match? True


In [49]:
# 3️⃣ Write all three out
aligned[0].rio.to_raster("LAI_aligned.tif")
aligned[1].rio.to_raster("CHM_aligned.tif")
aligned[2].rio.to_raster("LST_aligned.tif")

In [51]:
aligned

[<xarray.DataArray (y: 15184, x: 13807)> Size: 839MB
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
 Coordinates:
     spatial_ref  int64 8B 0
   * x            (x) float64 110kB 7.525e+05 7.525e+05 ... 7.533e+05 7.533e+05
   * y            (y) float64 121kB 4.516e+06 4.516e+06 ... 4.515e+06 4.515e+06
 Attributes:
     AREA_OR_POINT:       Area
     STATISTICS_MINIMUM:  0
     STATISTICS_MAXIMUM:  4.4462170600891
     STATISTICS_MEAN:     -9999
     STATISTICS_STDDEV:   -9999
     scale_factor:        1.0
     add_offset:          0.0
     long_name:           LTARgatesburg_2022Aug03_AltumSR_RGBnirRE_WGS84_UTM17...,
 <xarray.DataArray (y: 15184, x: 13807)> Size: 839MB
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..

### reproject rasters to 4326

In [26]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

def reproject_to_wgs84(src_path, dst_path):
    dst_crs = "EPSG:4326"
    with rasterio.open(src_path) as src:
        # Compute the transform & output shape for the new CRS
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        # Update metadata for the destination
        kwargs = src.meta.copy()
        kwargs.update({
            "crs": dst_crs,
            "transform": transform,
            "width": width,
            "height": height
        })
        
        # Open destination and reproject band by band
        with rasterio.open(dst_path, "w", **kwargs) as dst:
            for band in src.indexes:
                reproject(
                    source      = rasterio.band(src, band),
                    destination = rasterio.band(dst, band),
                    src_transform = src.transform,
                    src_crs       = src.crs,
                    dst_transform = transform,
                    dst_crs       = dst_crs,
                    resampling    = Resampling.bilinear,
                    dst_nodata    = src.nodata
                )

# run for each raster
reproject_to_wgs84("LAI_aligned.tif", "LAI_aligned_wgs84.tif")
reproject_to_wgs84("CHM_aligned.tif", "CHM_aligned_wgs84.tif")
reproject_to_wgs84("LST_aligned.tif", "LST_aligned_wgs84.tif")

# Applying HRMET

In [28]:
import math
import rasterio
from rasterio.windows import Window
import numpy as np
from datetime import datetime

# ─── 1) Your pixel-wise function (wraps HRMET_py) ───────────────
def pixel_block(lai_block, h_block, ts_block, lon_block, lat_block,
                datetime_, daylightSavings,
                Tair, SWin, u, ea, pa,
                Zair, Zu,
                albSoil, albVeg, emissSoil, emissVeg):
    # lai_block, h_block, ts_block, lon_block, lat_block are small 2D numpy arrays
    # We’ll vectorize HRMET_py over that block:
    out = np.empty_like(lai_block, dtype="float32")
    rows, cols = lai_block.shape
    for i in range(rows):
        for j in range(cols):
            out[i,j] = HRMET_py(
                LAI=float(lai_block[i,j]), h=float(h_block[i,j]), Tsurf=float(ts_block[i,j]),
                lon=float(lon_block[i,j]), lat=float(lat_block[i,j]),
                datetime_=datetime_, daylightSavings=daylightSavings,
                Tair=Tair, SWin=SWin, u=u, ea=ea, pa=pa,
                Zair=Zair, Zu=Zu,
                albSoil=albSoil, albVeg=albVeg,
                emissSoil=emissSoil, emissVeg=emissVeg
            )
    return out

# ─── 2) Open your three aligned rasters ──────────────────────────
src1 = rasterio.open("LAI_aligned_wgs84.tif")   # contains LAI
src2 = rasterio.open("CHM_aligned_wgs84.tif")   # contains canopy height (h)
src3 = rasterio.open("LST_aligned_wgs84.tif")   # contains surface temp (Tsurf)

# ─── 3) Prepare output file ─────────────────────────────────────
meta = src1.meta.copy()
meta.update({"dtype":"float32", "nodata": None})
dst = rasterio.open("pixelwise_output.tif", "w", **meta)

# ─── 4) Precompute lon/lat for every pixel (once) ───────────────
#    We’ll read them per-block anyway, but step this out so blocks know coords:
transform = src1.transform
width, height = src1.width, src1.height

# create 2D arrays of lon, lat for the whole grid
cols = np.arange(width)
rows = np.arange(height)
xs, ys = np.meshgrid(cols, rows)

lon_flat, lat_flat = rasterio.transform.xy(
    transform,
    ys.ravel(),    # flatten row indices
    xs.ravel(),    # flatten column indices
    offset="center"
)

# Reshape back into 2D arrays
lons = np.array(lon_flat, dtype=np.float32).reshape((height, width))
lats = np.array(lat_flat, dtype=np.float32).reshape((height, width))

# ─── 5) Decide your block size ───────────────────────────────────
#    Smaller → lower memory per-block, but more overhead loops.
block_width  = 512
block_height = 512
n_cols = math.ceil(width  / block_width)
n_rows = math.ceil(height / block_height)
total_blocks = n_cols * n_rows

# ─── 6) Loop over blocks with a simple progress print ────────────
#    Also define all your constants here:
datetime_        = datetime(2022,8,3,13,0)
daylightSavings = 1
Tair = EC_tower_data.loc['2022-08-03 13:00:00']['TA_1_1_1']
SWin = EC_tower_data.loc['2022-08-03 13:00:00']['SW_IN_1_1_1']
u    = EC_tower_data.loc['2022-08-03 13:00:00']['WS']
RH   = EC_tower_data.loc['2022-08-03 13:00:00']['RH_1_1_1']
esat = 0.6108 * np.exp(17.27 * Tair / (Tair + 237.3))   # saturation vapor pressure (kPa)
ea   = (RH / 100.0) * esat
pa   = EC_tower_data.loc['2022-08-03 13:00:00']['PA']
# based on measurement height from ameriflux website
Zair = 3
Zu   = 5

count = 0
for i in range(n_rows):
    for j in range(n_cols):
        count += 1
        # define window in pixel coords
        row_off = i * block_height
        col_off = j * block_width
        win_h = min(block_height, height - row_off)
        win_w = min(block_width,  width  - col_off)
        window = Window(col_off, row_off, win_w, win_h)

        # read the three bands for this window
        lai_block = src1.read(1, window=window)
        h_block   = src2.read(1, window=window)
        ts_block  = src3.read(1, window=window)
        
        # ─── clamp the canopy‐height so it never exceeds 2.5 ─────────────
        h_block = np.minimum(h_block, 2.5)

        # slice the precomputed lon/lat
        lon_block = lons[row_off:row_off+win_h, col_off:col_off+win_w]
        lat_block = lats[row_off:row_off+win_h, col_off:col_off+win_w]

        # compute
        out_block = pixel_block(
            lai_block, h_block, ts_block, lon_block, lat_block,
            datetime_, daylightSavings,
            Tair, SWin, u, ea, pa,
            Zair, Zu,
            albSoil, albVeg, emissSoil, emissVeg
        )

        # write back
        dst.write(out_block, 1, window=window)

        # print progress
        pct = count/total_blocks*100
        print(f"Block {count}/{total_blocks} ({pct:.1f}%)", end="\r")

print("\nDone.")
dst.close()
src1.close()
src2.close()
src3.close()

Block 891/891 (100.0%)
Done.


In [8]:
chunks = {"y":1024, "x":1024}
LAI_raster = rxr.open_rasterio("LAI_aligned.tif", chunks=chunks, masked=True).squeeze("band", drop=True).rio.reproject("EPSG:4326").rename("LAI")
CHM_raster = rxr.open_rasterio("CHM_aligned.tif", chunks=chunks, masked=True).squeeze("band", drop=True).rio.reproject("EPSG:4326").rename("CHM")
LST_raster = rxr.open_rasterio("LST_aligned.tif", chunks=chunks, masked=True).squeeze("band", drop=True).rio.reproject("EPSG:4326").rename("LST")

In [11]:
ds_ = xr.merge([LAI_raster, CHM_raster, LST_raster])

# repalce pixels with canopy h > 2.5 as h=2.5m
ds = ds_.copy()
ds["CHM"] = ds["CHM"].clip(max=2.5)

In [ ]:
albSoil = 0.105
albVeg = 0.2
emissSoil = 0.945
emissVeg = 0.94

In [16]:
# your constant inputs
datetime_        = datetime.strptime('2022-08-03 13:00:00', "%Y-%m-%d %H:%M:%S")
daylightSavings  = 1

Tair = EC_tower_data.loc['2022-08-03 13:00:00']['TA_1_1_1']
SWin = EC_tower_data.loc['2022-08-03 13:00:00']['SW_IN_1_1_1']
u    = EC_tower_data.loc['2022-08-03 13:00:00']['WS']
RH   = EC_tower_data.loc['2022-08-03 13:00:00']['RH_1_1_1']
esat = 0.6108 * np.exp(17.27 * Tair / (Tair + 237.3))   # saturation vapor pressure (kPa)
ea   = (RH / 100.0) * esat
pa   = EC_tower_data.loc['2022-08-03 13:00:00']['PA']

Zair, Zu        = 3.0, 5.0
albSoil, albVeg = 0.105, 0.2
emissSoil, emissVeg = 0.945, 0.94

vec_hrmet = np.vectorize(
    lambda lai, h, ts, lon, lat: HRMET_py(
        datetime_=datetime_,
        daylightSavings=daylightSavings,
        longitude=float(lon), latitude=float(lat),
        Tair=Tair, SWin=SWin, u=u, ea=ea, pa=pa,
        Zair=Zair, Zu=Zu,
        LAI=float(lai), h=float(h), Tsurf=float(ts),
        albSoil=albSoil, albVeg=albVeg,
        emissSoil=emissSoil, emissVeg=emissVeg
    ),
    otypes=[float]
)

In [ ]:
# Build X, Y coordinate DataArrays
X_da, Y_da = xr.broadcast(ds.x, ds.y)
X_da.name = "lon"
Y_da.name = "lat"

# Apply over the grid, passing constants in kwargs
result = xr.apply_ufunc(
    vec_hrmet,
    ds.LAI, ds.CHM, ds.LST,  # the three raster layers
    X_da, Y_da,                 # pixel coords
    input_core_dims=[["y","x"]]*5,   # each arg is 2D
    output_core_dims=[["y","x"]],    # result is 2D
    dask="parallelized",             # leverages your chunks
    output_dtypes=[float],
    vectorize=False                  # pixel‐wise in C
)

# Wrap into a Dataset
out_ds = result.to_dataset(name="calc_output")
out_ds = out_ds.assign_coords(x=ds.x, y=ds.y)
out_ds.rio.write_crs(ds.rio.crs, inplace=True)

# (Optional) write to GeoTIFF
with ProgressBar():
    out_ds.rio.to_raster("pixel_calc_output.tif")